In [1]:
(ns interpreter)
(def car first)
(def cdr rest)
(defn cadr [x] (car (cdr x)))
(defn caddr [x] (car (cdr (cdr x))))
(defn cadddr [x] (car (cdr (cdr (cdr x)))))
(defn cddr [x] (cdr (cdr x)))
(defn cadr [x] (car (cdr x)))

(defn mput [m key value] (do (. m put key value) m))
(defn todict [m] (into {} m))


#'interpreter/todict

In [2]:
(import (java.util HashMap))
;; (def ih2 (HashMap.))

java.util.HashMap

In [9]:
(defn tag-check [e sym] (and (list? e) (= (car e) sym)))
(defn sum? [e] (tag-check e 'plus*))
(defn define? [exp] (tag-check exp 'define*))
(defn application? [e] (list? e))
(defn lambda? [e] (tag-check e 'lambda*))

(defn eval [exp env]
  (cond
    (number? exp) exp
    ;; (sum? exp) (eval-sum exp)
    ;; part2
    (symbol? exp) (lookup exp env)
    (define? exp) (eval-define exp env)
    ;; part3
    ;; (greater? exp) (eval-greater exp)
    (if? exp) (eval-if exp env)
    (lambda? exp) (eval-lambda exp env)
    (application? exp) (appli (eval (car exp)) (map eval (cdr exp)))
    ;; part4 
    ;; 1. (eval (car exp)) will extract the stored op from table
    ;; 2. all of the arguments should be in the final status.
    true (throw (Exception. (str exp)))))



;; 2. names
(def table-put! mput)
(def table-get get)
(def GV (HashMap.))


(defn lookup [name env]
  (let [value (table-get env name)]
    (if (nil? value)
      (throw (Exception. (str "unbound variable: " name)))
      value)))

(defn eval-define [exp env]
  (let [name (cadr exp)
        defined-to-be (caddr exp)]
    (table-put! env name (eval defined-to-be env))
    'undefined))
;; 3. conditionals and if
;; (defn greater? [exp] (tag-check exp 'greater*))
;; (defn if? [exp] (tag-check exp 'if*))

(defn eval-if [exp env]
  (let [predicate (cadr exp)
        consequent (caddr exp)
        alternative (cadddr exp)]
    (let [test (eval predicate)]
      (cond
        (= true test) (eval consequent env)
        (= false test) (eval alternative env)
        true (throw (Exception. (str "predicate not a conditional: " predicate)))))))

;; 4. stored op

(def prim-tag 'primitive)
(defn primitive? [e] (tag-check e prim-tag))
(defn make-primitive [sch-proc] (list prim-tag sch-proc))
(defn get-sch-proc [prim] (cadr prim))

;; (defn apply [opt operands]
;;   (if (primitive? opt)
;;     (sch-apply (get-sch-proc opt) operands)
;;     (throw (Exception. (str "operator not a procedure " opt " " operands)))))

(defn appli [op operands]
  ;; (println op)
  ;; (println operands)
  (if (primitive? op)
    (apply (get-sch-proc op) operands)
    (throw (Exception.))))

(defn eval-lambda [exp env] 
  (let [args (cadr exp) 
    body (caddr exp)]
    (make-compound args body env)))

(table-put! GV 'plus* (make-primitive +))
(table-put! GV 'greater* (make-primitive >))
(table-put! GV 'true* true)


{plus* (primitive #function[clojure.core/+]), true* true, greater* (primitive #function[clojure.core/>])}

In [10]:
(def exp '(plus* 24 (plus* 5 6)))
;; (eval exp)
;; names
;; (eval '(define* x* (plus* 4 5)))
;; (eval '(plus* x* 2))

;; conditionals and if
;; (eval '(define* y* 9))
;; (eval '(if* (greater* y* 6) (plus* y* 2) 13))

;; (eval '(define* z* 9) GV)
;; (eval '(plus* 9 6))
(apply (get-sch-proc (eval 'plus* GV)) (list 1 2))
;; (eval '(if* true* 13 2))
;; (eval ax)

3